# 6.2 Стекинг

In [26]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.base import clone

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from matplotlib import pyplot as plt

%matplotlib inline

In [27]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/covtype.data.gz', sep=',', header=None)[:10000]

In [28]:
df.head(3)

,0,1,2,3,4,5,6,7,8,9,...,45,46,47,48,49,50,51,52,53,54
0,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,2


In [29]:
features = list(range(0, 54))
target = 54

df = df[(df[target] == 1) | (df[target] == 2)]

In [30]:
cover_train, cover_test = train_test_split(df, test_size=0.5)

cover_X_train, cover_y_train = cover_train[features], cover_train[target]
cover_X_test, cover_y_test = cover_test[features], cover_test[target]

In [31]:
scaler = StandardScaler()
cover_X_train = scaler.fit_transform(cover_X_train)
cover_X_test = scaler.transform(cover_X_test)

Stacking — еще один способ объединить несколько алгоритмов в один, который часто используется как в решении реальных задач из промышленной сферы, так и в конкурсах на платформах вроде Kaggle.  
Подход использует понятие _базовых классификаторов_, каждый из которых независимо обучается на некотором (возможно одном и том же) множестве признаков, а также _мета-классификатора_, использующего предсказания базовых классификаторов как признаки.

Для избежания переобучения будем разбивать обучающую выборку на фолды.  
Например, фолды при разбиении на три части:  
``==*``  
``=*=``  
``*==``  

Это требуется для того, чтобы получить новые признаки (ответы алгоритмов на первом уровне) на всей обучающей выборке, т.е. ответы алгоритма на тех объектах, которые не были использованы во время обучения. В примере выше мы будем использовать ответы алгоритма, полученные на объектах звездочках. _Важно_: на каждом фолде мы обучаем алгоритм заново.

In [32]:
def compute_meta_feature(clf, X_train, X_test, y_train, cv):
    """
    Computes meta-features using the classifier.
    
    :arg clf: scikit-learn classifier
    :args X_train, y_train: training set
    :arg X_test: testing set
    :arg cv: cross-validation folding
    """
    X_meta_train = np.zeros_like(y_train, dtype=np.float32)
    for train_fold_index, predict_fold_index in cv.split(X_train):
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]
        
        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)
        X_meta_train[predict_fold_index] = folded_clf.predict_proba(X_fold_predict)[:, 1]
    
    meta_clf = clone(clf)
    meta_clf.fit(X_train, y_train)
    
    X_meta_test = meta_clf.predict_proba(X_test)[:, 1]
    
    return X_meta_train, X_meta_test

In [33]:
def generate_meta_features(classifiers, X_train, X_test, y_train, cv):
    """
    Generates metafeatures using a list of classifiers.
    
    :arg classifiers: list of scikit-learn classifiers
    :args X_train, y_train: training set
    :arg X_test: testing set
    :arg cv: cross-validation folding
    """
    features = [
        compute_meta_feature(clf, X_train, X_test, y_train, cv)
        for clf in tqdm(classifiers)
    ]
    
    stacked_features_train = np.vstack([
        features_train for features_train, features_test in features
    ]).T

    stacked_features_test = np.vstack([
        features_test for features_train, features_test in features
    ]).T
    
    return stacked_features_train, stacked_features_test

In [35]:
clf = GradientBoostingClassifier(n_estimators=300)
clf.fit(cover_X_train, cover_y_train)

accuracy_score(clf.predict(cover_X_test), cover_y_test)

0.7963354474982381

In [36]:
cv = KFold(n_splits=10, shuffle=True)

stacked_features_train, stacked_features_test = generate_meta_features([
    LogisticRegression(C=0.001, penalty='l1', solver='liblinear', max_iter=5000),
    LogisticRegression(C=0.001, penalty='l2', solver='liblinear', max_iter=5000),  
    RandomForestClassifier(n_estimators=300, n_jobs=-1),
    GradientBoostingClassifier(n_estimators=300)
], cover_X_train, cover_X_test, cover_y_train.values, cv)

100%|██████████| 4/4 [00:24<00:00,  6.10s/it]


In [37]:
total_features_train = np.hstack([cover_X_train, stacked_features_train])
total_features_test = np.hstack([cover_X_test, stacked_features_test])

In [38]:
np.random.seed(42)
clf = LogisticRegression(penalty='none', solver='lbfgs')
clf.fit(stacked_features_train, cover_y_train)
accuracy_score(clf.predict(stacked_features_test), cover_y_test)

d:\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(


0.8040873854827343

In [39]:
def compute_meta_feature_mean(clf, X_train, X_test, y_train, cv):
    """
    Эта функция подсчитывает признаки для мета-классификатора. 
    Они являются вероятностями классов при решении задачи многоклассовой классификации.

    :arg clf: классификатор
    :args X_train, y_train: обучающая выборка
    :arg X_test: признаки тестовой выборки
    :arg cv: класс, генерирующий фолды (KFold)

    :returns X_meta_train, X_meta_test: новые признаки для обучающей и тестовой выборок
    """
# Напишите ваш код ниже
    n_classes = len(np.unique(y_train))
    X_meta_test = np.zeros((len(X_test), n_classes), dtype=np.float32)
    X_meta_test = np.zeros(len(X_test), dtype=np.float32)
    splits = 0
    
    for train_fold_index, predict_fold_index in cv.split(X_train):
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]
        splits += 1

        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)

        X_meta_train[predict_fold_index] = folded_clf.predict_proba(X_fold_predict)
        X_meta_test[predict_fold_index] = meta_clf.predict_proba(X_test)

    meta_clf = clone(clf)
    meta_clf.fit(X_train, y_train)

    X_meta_test = meta_clf.predict_proba(X_test)
    X_meta_test = sum(X_meta_tests_array) / splits

    return X_meta_train, X_meta_test

Задание 6.6.2

In [51]:
import pandas as pd
import numpy as np

from sklearn.ensemble import (AdaBoostClassifier, GradientBoostingClassifier,
                              RandomForestClassifier, ExtraTreesClassifier)
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import clone
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.datasets import load_digits

from tqdm import tqdm

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats.distributions import randint

In [52]:
dataset = load_digits()
X, y = dataset['data'], dataset['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [71]:
def compute_meta_feature(clf, X_train, X_test, y_train, cv):
    
    n_classes = len(np.unique(y_train))
    X_meta_train = np.zeros((len(y_train), n_classes), dtype=np.float32)

    splits = cv.split(X_train, y_train)
    for train_fold_index, predict_fold_index in splits:
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]
        
        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)
        
        X_meta_train[predict_fold_index] = folded_clf.predict_proba(X_fold_predict)
    
    meta_clf = clone(clf)
    meta_clf.fit(X_train, y_train)
    
    X_meta_test = meta_clf.predict_proba(X_test)
    
    return X_meta_train, X_meta_test

In [54]:
def generate_meta_features(classifiers, X_train, X_test, y_train, cv):
   
    features = [
        compute_meta_feature(clf, X_train, X_test, y_train, cv)
        for clf in tqdm(classifiers)
    ]
    
    stacked_features_train = np.hstack([
        features_train for features_train, features_test in features
    ])

    stacked_features_test = np.hstack([
        features_test for features_train, features_test in features
    ])
    
    return stacked_features_train, stacked_features_test

In [55]:
cv = KFold(n_splits=10, shuffle=True, random_state=42)

def compute_metric(clf, X_train=X_train, y_train=y_train, X_test=X_test):
    clf.fit(X_train, y_train)
    y_test_pred = clf.predict(X_test)
    return np.round(f1_score(y_test, y_test_pred, average='macro'), 6)

Задание 6.6.2

Используйте функцию generate_meta_features для стекинга следующих алгоритмов:
логистическая регрессия с L1-регуляризацией, C=0.001, солвер — 'saga', схема работы мультиклассовой классификации — one-vs-rest, максимальное допустимое количество итераций — 2000
логистическая регрессия с L2-регуляризацией, C=0.001, солвер — 'saga', схема работы мультиклассовой классификации — multinomial, максимальное допустимое количество итераций — 2000
случайный лес из 300 деревьев
градиентный бустинг из 200 деревьев
Как мета-алгоритм используйте логистическую регрессию без регуляризации со схемой работы мультиклассовой классификации — auto и солвером 'lbfgs'.
Посчитайте качество при помощи передачи новых признаков в функцию compute_metric.

In [ ]:
np.random.seed(42)

In [56]:
estimators = [
    LogisticRegression(penalty='l1', C=0.001, solver='saga', multi_class='ovr', max_iter=2000),
    LogisticRegression(penalty='l2', C=0.001, solver='saga', multi_class='multinomial', max_iter=2000),
    RandomForestClassifier(n_estimators=300),
    GradientBoostingClassifier(n_estimators=200)
]
meta_lr = LogisticRegression(penalty=None, solver='lbfgs', multi_class='auto')

In [57]:
S_X_train, S_X_test = generate_meta_features(estimators, X_train, X_test, y_train, cv)
print(compute_metric(meta_lr, X_train=S_X_train, X_test=S_X_test))

100%|██████████| 4/4 [03:43<00:00, 55.79s/it]

0.97591


Задание 6.6.3

Используйте функцию generate_meta_features для стекинга следующих алгоритмов:
случайный лес из 300 деревьев
случайный лес из 200 экстремальных деревьев
Как мета-алгоритм используйте логистическую регрессию без регуляризации со схемой работы мультиклассовой классификации — auto и солвером 'lbfgs'.
Посчитайте качество при помощи передачи новых признаков в функцию compute_metric.

In [60]:
estimators = [
    RandomForestClassifier(n_estimators=300),
    ExtraTreesClassifier(n_estimators=200)
]
meta_lr = LogisticRegression(penalty=None, solver='lbfgs', multi_class='auto')

S_X_train, S_X_test = generate_meta_features(estimators, X_train, X_test, y_train, cv)
print(compute_metric(meta_lr, X_train=S_X_train, X_test=S_X_test))

100%|██████████| 2/2 [00:19<00:00,  9.63s/it]

0.980501


Задание 6.6.4

Используйте функцию generate_meta_features для стекинга следующих алгоритмов:
метод ближайшего соседа (k-NN) со стандартными параметрами
случайный лес из 300 экстремальных деревьев
Как мета-алгоритм используйте логистическую регрессию без регуляризации со схемой работы мультиклассовой классификации — auto и солвером 'lbfgs'.
Посчитайте качество при помощи передачи новых признаков в функцию compute_metric.

In [49]:
estimators = [
    KNeighborsClassifier(),
    ExtraTreesClassifier(n_estimators=300)
]
meta_lr = LogisticRegression(penalty=None, solver='lbfgs', multi_class='auto')

S_X_train, S_X_test = generate_meta_features(estimators, X_train, X_test, y_train, cv)
print(compute_metric(meta_lr, X_train=S_X_train, X_test=S_X_test))

100%|██████████| 2/2 [00:12<00:00,  6.10s/it]

0.989752



d:\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Задание 6.6.5

Используйте функцию generate_meta_features для стекинга следующих алгоритмов:
логистическая регрессия с L1-регуляризацией, C=0.001, солвер — 'saga', схема работы мультиклассовой классификации — one-vs-rest, максимальное допустимоей количество итераций — 2000
метод ближайшего соседа со стандартными параметрами
случайный лес из 300 экстремальных деревьев
AdaBoost со стандартными параметрами
Как мета-алгоритм используйте логистическую регрессию без регуляризации со схемой работы мультиклассовой классификации — auto и солвером 'lbfgs'.
Посчитайте качество при помощи передачи новых признаков в функцию compute_metric.

In [59]:
estimators = [
    LogisticRegression(penalty='l1', C=0.001, solver='saga', multi_class='ovr', max_iter=2000),
    KNeighborsClassifier(),
    ExtraTreesClassifier(n_estimators=300),
    AdaBoostClassifier()
]
meta_lr = LogisticRegression(penalty=None, solver='lbfgs', multi_class='auto')

S_X_train, S_X_test = generate_meta_features(estimators, X_train, X_test, y_train, cv)
print(compute_metric(meta_lr, X_train=S_X_train, X_test=S_X_test))

100%|██████████| 4/4 [00:55<00:00, 13.95s/it]

0.989752


Задание 6.6.6

In [63]:
estimators = [
    RandomForestClassifier(n_estimators=300),
    ExtraTreesClassifier(n_estimators=300)
]
meta_lr = LogisticRegression(penalty=None, solver='lbfgs', multi_class='auto')

S_X_train, S_X_test = generate_meta_features(estimators, X_train, X_test, y_train, cv)
print(compute_metric(meta_lr, X_train=S_X_train, X_test=S_X_test))

100%|██████████| 2/2 [00:45<00:00, 22.87s/it]

0.982341


Задание 6.6.7

В предыдущей задаче измените 10 фолдов на 20. Укажите полученное качество.

In [ ]:
estimators = [
    RandomForestClassifier(n_estimators=300),
    ExtraTreesClassifier(n_estimators=300)
]
meta_lr = LogisticRegression(penalty=None, solver='lbfgs', multi_class='auto')

cv = KFold(n_splits=20, shuffle=True, random_state=42)
S_X_train, S_X_test = generate_meta_features(estimators, X_train, X_test, y_train, cv)
print(compute_metric(meta_lr, X_train=S_X_train, X_test=S_X_test))

Задание 6.6.8

В предыдущей задаче укажите количество фолдов равным 5 и поменяйте мета-алгоритм на случайный лес со стандартными параметрами. Укажите полученное качество.

In [67]:
estimators = [
    RandomForestClassifier(n_estimators=300),
    ExtraTreesClassifier(n_estimators=300)
]
meta_lr = RandomForestClassifier()

cv = KFold(n_splits=5, shuffle=True, random_state=42)
S_X_train, S_X_test = generate_meta_features(estimators, X_train, X_test, y_train, cv)
print(compute_metric(meta_lr, X_train=S_X_train, X_test=S_X_test))

100%|██████████| 2/2 [00:11<00:00,  5.87s/it]


0.982325


Задание 6.6.9

В предыдущей задаче поменяйте мета-алгоритм на метод ближайших соседей (k-NN) со стандартными параметрами. Укажите полученное качество.

In [65]:
estimators = [
    RandomForestClassifier(n_estimators=300),
    ExtraTreesClassifier(n_estimators=300)
]
meta_lr = KNeighborsClassifier()

cv = KFold(n_splits=5, shuffle=True, random_state=42)
S_X_train, S_X_test = generate_meta_features(estimators, X_train, X_test, y_train, cv)
print(compute_metric(meta_lr, X_train=S_X_train, X_test=S_X_test))

100%|██████████| 2/2 [00:13<00:00,  6.73s/it]

0.98762


Задание 6.6.10

В предыдущей задаче поменяйте мета-алгоритм на градиентный бустинг со стандартными параметрами. Укажите полученное качество.

In [72]:
estimators = [
    RandomForestClassifier(n_estimators=300),
    ExtraTreesClassifier(n_estimators=300)
]
meta_lr = GradientBoostingClassifier()

cv = KFold(n_splits=5, shuffle=True, random_state=42)
S_X_train, S_X_test = generate_meta_features(estimators, X_train, X_test, y_train, cv)
print(compute_metric(meta_lr, X_train=S_X_train, X_test=S_X_test))







100%|██████████| 2/2 [00:07<00:00,  3.79s/it]

0.98762


Задание 6.6.11

In [ ]:
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

In [ ]:
estimators = [
    RandomForestClassifier(n_estimators=300, criterion='gini', max_depth=24),
    ExtraTreesClassifier(n_estimators=300)
]
meta_lr = ExtraTreesClassifier(n_estimators=100)

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
S_X_train, S_X_test = generate_meta_features(estimators, X_train, X_test, y_train, cv)
print(compute_metric(meta_lr, X_train=S_X_train, X_test=S_X_test))

Задание 6.6.12

Обучите на тренировочной выборке следующие алгоритмы:
случайный лес из 300 деревьев, критерий Джини, максимальная глубина — 24
случайный лес из 300 экстремальных деревьев
логистическую регрессию со стандартными параметрами
Усредните их ответы на тестовой выборке и посчитайте качество аналогично функции compute_metric (F1-score с макро-усреднением, округленный до 6 знака).

In [100]:
y_pred_1 = RandomForestClassifier(n_estimators=300, criterion='gini', max_depth=24).fit(X_train, y_train).predict(X_test)
y_pred_2 = ExtraTreesClassifier(n_estimators=100).fit(X_train, y_train).predict(X_test)
y_pred_3 = LogisticRegression().fit(X_train, y_train).predict(X_test)
y_test_pred = np.round((y_pred_1 + y_pred_2 + y_pred_3) / 3).astype(int)

d:\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [101]:
np.round(f1_score(y_test, y_test_pred, average='macro'), 6)

0.961905